In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn import preprocessing

def print_unique_values(df,coln):
    arr=np.unique(df[coln].to_list())
    for i in range(len(arr)):
        print(i,"--",arr[i])
    del arr
def concat_feat_val(df,coln,value_arr,new_value):
    df[coln] = df[coln].replace(value_arr,new_value)
def missing_val(df,coln,val):
    df[coln]=df[coln].fillna(val)

1. Download the data and clean it. It 600mb uncompressed data


In [2]:
#https://files.consumerfinance.gov/ccdb/complaints.csv.zip
# df=df.drop(["Unnamed: 0","Complaint ID","Timely response?","Date sent to company","Tags","Submitted via","Company public response","Consumer consent provided?","Sub-issue"],axis=1)
# df.to_csv("final_clean_consumer.csv")

2. read the cleaned dataset

In [4]:
!ls ..

AISC2000ApplicationExercise2.docx alpha_king
AISC2000ApplicationExercise2.pdf  complaint_1.ipynb
Consumer_complaint                complaints.csv
Data_Clean_1.ipynb                consumer-complaints.tfrecord
InClass_qasim.csv                 final_clean_consumer.csv
Untitled.ipynb                    final_data.csv
Untitled1.ipynb                   iris.csv
Untitled2.ipynb                   qasim_final
Untitled3.ipynb                   qasim_final_2
Untitled4.ipynb                   rough_code.ipynb


In [9]:
df=pd.read_csv("../complaints.csv")

/var/folders/k4/29bygr_s3mv0rn4yh7_rvtmm0000gn/T/ipykernel_4246/2118662780.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("../complaints.csv")


In [11]:
df.head(3)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2018-07-31,Checking or savings account,Savings account,Opening an account,Unable to open an account,NaN,Company believes the complaint is the result o...,"Bank of CFPB, Washington, DC CFPB PIV",NaN,NaN,NaN,NaN,Web,2018-07-31,Closed with explanation,Yes,NaN,3527315
1,2020-10-27,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,NaN,NaN,HEARTLAND PAYMENT SYSTEMS INC,NJ,7060.0,NaN,NaN,Fax,2020-10-27,Closed with explanation,Yes,NaN,4491254
2,2018-08-02,Mortgage,Conventional home mortgage,Closing on a mortgage,NaN,NaN,Company believes the complaint is the result o...,"Bank of CFPB, Washington, DC CFPB PIV",NaN,NaN,NaN,NaN,Web,2018-08-02,Closed with explanation,Yes,NaN,3527438


In [19]:
x=df[df["Consumer complaint narrative"].notna()]
x[x["Consumer disputed?"].notna()].isna().sum()

Date received                        0
Product                              0
Sub-product                      52172
Issue                                0
Sub-issue                        80990
Consumer complaint narrative         0
Company public response          85940
Company                              0
State                              459
ZIP code                           449
Tags                            137063
Consumer consent provided?           0
Submitted via                        0
Date sent to company                 0
Company response to consumer         0
Timely response?                     0
Consumer disputed?                   0
Complaint ID                         0
dtype: int64

In [21]:
x[x["Consumer disputed?"].notna()].shape

(164062, 18)

In [10]:
df=df.drop("Unnamed: 0",axis=1)

KeyError: "['Unnamed: 0'] not found in axis"

In [ ]:
df.head(5)

In [8]:
df.describe(include=object).T

,count,unique,top,freq
Date received,768457,1970,2017-01-19,2070
Product,768457,13,Mortgage,226897
Sub-product,533294,50,Other mortgage,86635
Issue,768457,99,"Loan modification,collection,foreclosure",112309
Consumer complaint narrative,164062,160968,I am filing this complaint because Experian ha...,103
Company,768457,4284,"BANK OF AMERICA, NATIONAL ASSOCIATION",65992
State,762807,62,CA,110638
ZIP code,762810,44609,48382,839
Company response to consumer,768457,7,Closed with explanation,577955
Consumer disputed?,768457,2,No,620079


In [158]:
print("Missing values")
print("")
print(df.isnull().sum())
print("")
print("Missing values in %")
print("")
print((df.isnull().sum()/df.shape[0])*100)

Missing values

Date received                        0
Product                              0
Sub-product                     235163
Issue                                0
Consumer complaint narrative    604395
Company                              0
State                             5650
ZIP code                          5647
Company response to consumer         0
Consumer disputed?                   0
dtype: int64

Missing values in %

Date received                    0.000000
Product                          0.000000
Sub-product                     30.601973
Issue                            0.000000
Consumer complaint narrative    78.650465
Company                          0.000000
State                            0.735240
ZIP code                         0.734849
Company response to consumer     0.000000
Consumer disputed?               0.000000
dtype: float64


3. Reading unique values in particular column

In [159]:
print_unique_values(df,"Product")

0 -- Bank account or service
1 -- Checking or savings account
2 -- Consumer Loan
3 -- Credit card
4 -- Credit reporting
5 -- Debt collection
6 -- Money transfers
7 -- Mortgage
8 -- Other financial service
9 -- Payday loan
10 -- Prepaid card
11 -- Student loan
12 -- Virtual currency


In [160]:
print_unique_values(df,"Sub-product")

0 -- (CD) Certificate of deposit
1 -- Auto
2 -- Cashing a check without an account
3 -- Check cashing
4 -- Checking account
5 -- Conventional adjustable mortgage (ARM)
6 -- Conventional fixed mortgage
7 -- Conventional home mortgage
8 -- Credit card
9 -- Credit repair
10 -- Debt settlement
11 -- Domestic (US) money transfer
12 -- Electronic Benefit Transfer / EBT card
13 -- FHA mortgage
14 -- Federal student loan
15 -- Federal student loan servicing
16 -- Foreign currency exchange
17 -- General purpose card
18 -- Gift or merchant card
19 -- Government benefit payment card
20 -- Home equity loan or line of credit
21 -- I do not know
22 -- ID prepaid card
23 -- Installment loan
24 -- International money transfer
25 -- Medical
26 -- Mobile wallet
27 -- Money order
28 -- Mortgage
29 -- Non-federal student loan
30 -- Other (i.e. phone, health club, etc.)
31 -- Other bank product/service
32 -- Other banking product or service
33 -- Other mortgage
34 -- Other special purpose card
35 -- Other 

In [161]:
missing_val(df,"Sub-product","Not given")

4. Mappping new values in Product & Sub-product coln

In [162]:
# Credits https://www.kaggle.com/code/ashwinids/cleaning-exploring-consumer-complaints-data
mapping_old2new = {
    "Auto": "Auto debt",
    "Credit card": "Credit card debt",
    "Federal student loan": "Federal student loan debt",
    "Medical": "Medical debt",
    "Mortgage": "Mortgage debt",
    "Non-federal student loan": "Private student loan debt",
    "Other (i.e. phone, health club, etc.)": "Other debt",
    "Payday loan": "Payday loan debt",
    "Non-federal student loan": "Private student loan",
    "Federal student loan servicing": "Federal student loan",
    "Credit repair": "Credit repair services",
    "Credit reporting": "Credit reporting",
    "Conventional adjustable mortgage (ARM)": "Conventional home mortgage",
    "Conventional fixed mortgage": "Conventional home mortgage",
    "Home equity loan or line of credit": "Home equity loan or line of credit (HELOC)",
    "Other": "Other type of mortgage",
    "Other mortgage": "Other type of mortgage",
    "Second mortgage":"Other type of mortgage",
    "Credit card": "General-purpose credit card or charge card",
    "General purpose card": "General-purpose prepaid card",
    "Gift or merchant card": "Gift card",
    "Electronic Benefit Transfer / EBT card": "Government benefit card",
    "Government benefit payment card": "Government benefit card",
    "ID prepaid card": "Student prepaid card",
    "Other special purpose card":  "Other prepaid card",
    "Store credit card": "Other prepaid card",
    "Transit card": "Other prepaid card",
    "(CD) Certificate of deposit": "CD (Certificate of Deposit)",
    "Other bank product/service": "Other banking product or service",
    "Cashing a check without an account": "Other banking product or service",
    "Vehicle lease": "Lease",
    "Vehicle loan": "Loan",
    "Check cashing": "Check cashing service",
    "Mobile wallet": "Mobile or digital wallet",
    "Traveler’s/Cashier’s checks": "Traveler's check or cashier's check"
}
prod2sub = {
    "Auto debt": "Debt collection",
    "Credit card debt": "Debt collection",
    "Federal student loan debt": "Debt collection",
    "I do not know": "Debt collection",
    "Medical debt": "Debt collection",
    "Mortgage debt": "Debt collection",
    "Private student loan debt": "Debt collection",
    "Other debt": "Debt collection",
    "Payday loan debt": "Debt collection",
    "Credit repair services": "Credit reporting, credit repair services, or other personal consumer reports",
    "Credit reporting": "Credit reporting, credit repair services, or other personal consumer reports",
    "Other personal consumer report": "Credit reporting, credit repair services, or other personal consumer reports",
    "Conventional home mortgage": "Mortgage",
    "FHA mortgage": "Mortgage",
    "Home equity loan or line of credit (HELOC)": "Mortgage",
    "Other type of mortgage": "Mortgage",
    "Reverse mortgage": "Mortgage",
    "VA mortgage": "Mortgage",
    "General-purpose credit card or charge card": "Credit card or prepaid card",
    "General-purpose prepaid card": "Credit card or prepaid card",
    "Gift card": "Credit card or prepaid card",
    "Government benefit card": "Credit card or prepaid card",
    "Student prepaid card": "Credit card or prepaid card",
    "Payroll card": "Credit card or prepaid card",
    "Other prepaid card": "Credit card or prepaid card",
    "CD (Certificate of Deposit)": "Checking or savings account",
    "Checking account": "Checking or savings account",
    "Other banking product or service": "Checking or savings account",
    "Savings account": "Checking or savings account",
    "Lease": "Vehicle loan or lease",
    "Loan": "Vehicle loan or lease",
    "Federal student loan": "Student loan",
    "Private student loan": "Student loan",
    "Installment loan": "Payday loan, title loan, or personal loan",
    "Pawn loan": "Payday loan, title loan, or personal loan",
    "Payday loan": "Payday loan, title loan, or personal loan",
    "Personal line of credit": "Payday loan, title loan, or personal loan",
    "Title loan": "Payday loan, title loan, or personal loan",
    "Check cashing service": "Money transfer, virtual currency, or money service",
    "Debt settlement": "Money transfer, virtual currency, or money service",
    "Domestic (US) money transfer": "Money transfer, virtual currency, or money service",
    "Foreign currency exchange": "Money transfer, virtual currency, or money service",
    "International money transfer": "Money transfer, virtual currency, or money service",
    "Mobile or digital wallet": "Money transfer, virtual currency, or money service",
    "Money order": "Money transfer, virtual currency, or money service",
    "Refund anticipation check": "Money transfer, virtual currency, or money service",
    "Traveler's check or cashier's check": "Money transfer, virtual currency, or money service",
    "Virtual currency": "Money transfer, virtual currency, or money service",
    "Not given":"Not given"
}


In [163]:
# Credits https://www.kaggle.com/code/ashwinids/cleaning-exploring-consumer-complaints-data

def get_subprods(x):
    
    if x['Sub-product'] in mapping_old2new:
        if x['Sub-product']=="Other":
            if x['Product']=='Mortage':
                return("Other type of mortgage")
            else:
                return("Other debt")
        else:
            return(mapping_old2new[x['Sub-product']])
    else:
        return(x['Sub-product'])

df['Sub-product'] = df[['Product','Sub-product']].apply(lambda x: get_subprods(x), axis =1)

In [164]:
# Credits https://www.kaggle.com/code/ashwinids/cleaning-exploring-consumer-complaints-data

prodmap = {
    "Payday loan": "Payday loan, title loan, or personal loan",
    "Credit reporting": "Credit reporting, credit repair services, or other personal consumer reports",
    "Credit card": "Credit card or prepaid card"
}
def get_product(x):
    
    if not isinstance(x['Sub-product'], str):
        if x['Product'] in prodmap:
            return(prodmap[x['Product']])
        else:
            return(x['Product'])
    else:
        return(prod2sub[x['Sub-product']])
    
df['Product'] = df[['Product','Sub-product']].apply(lambda x: get_product(x), axis =1)


In [165]:
print_unique_values(df,"Product")

0 -- Checking or savings account
1 -- Credit card or prepaid card
2 -- Credit reporting, credit repair services, or other personal consumer reports
3 -- Debt collection
4 -- Money transfer, virtual currency, or money service
5 -- Mortgage
6 -- Not given
7 -- Payday loan, title loan, or personal loan
8 -- Student loan
9 -- Vehicle loan or lease


In [166]:
print_unique_values(df,"Sub-product")

0 -- Auto debt
1 -- CD (Certificate of Deposit)
2 -- Check cashing service
3 -- Checking account
4 -- Conventional home mortgage
5 -- Credit repair services
6 -- Debt settlement
7 -- Domestic (US) money transfer
8 -- FHA mortgage
9 -- Federal student loan
10 -- Federal student loan debt
11 -- Foreign currency exchange
12 -- General-purpose credit card or charge card
13 -- General-purpose prepaid card
14 -- Gift card
15 -- Government benefit card
16 -- Home equity loan or line of credit (HELOC)
17 -- I do not know
18 -- Installment loan
19 -- International money transfer
20 -- Lease
21 -- Loan
22 -- Medical debt
23 -- Mobile or digital wallet
24 -- Money order
25 -- Mortgage debt
26 -- Not given
27 -- Other banking product or service
28 -- Other debt
29 -- Other prepaid card
30 -- Other type of mortgage
31 -- Pawn loan
32 -- Payday loan debt
33 -- Payroll card
34 -- Personal line of credit
35 -- Private student loan
36 -- Refund anticipation check
37 -- Reverse mortgage
38 -- Savings ac

In [167]:
print_unique_values(df,"Issue")

0 -- APR or interest rate
1 -- Account opening, closing, or management
2 -- Account terms and changes
3 -- Adding money
4 -- Advertising and marketing
5 -- Advertising, marketing or disclosures
6 -- Application processing delay
7 -- Application, originator, mortgage broker
8 -- Applied for loan/did not receive money
9 -- Arbitration
10 -- Balance transfer
11 -- Balance transfer fee
12 -- Bankruptcy
13 -- Billing disputes
14 -- Billing statement
15 -- Can't contact lender
16 -- Can't repay my loan
17 -- Can't stop charges to bank account
18 -- Cash advance
19 -- Cash advance fee
20 -- Charged bank acct wrong day or amt
21 -- Charged fees or interest I didn't expect
22 -- Closing/Cancelling account
23 -- Collection debt dispute
24 -- Collection practices
25 -- Communication tactics
26 -- Cont'd attempts collect debt not owed
27 -- Convenience checks
28 -- Credit card protection / Debt protection
29 -- Credit decision / Underwriting
30 -- Credit determination
31 -- Credit line increase/de

In [168]:
df.isna().sum()

Date received                        0
Product                              0
Sub-product                          0
Issue                                0
Consumer complaint narrative    604395
Company                              0
State                             5650
ZIP code                          5647
Company response to consumer         0
Consumer disputed?                   0
dtype: int64

In [169]:
missing_val(df,"State","Not given")

In [170]:
df["ZIP code"].describe()

count     762810
unique     44609
top        48382
freq         839
Name: ZIP code, dtype: object

In [171]:
missing_val(df,"ZIP code",0)

In [172]:
df=pd.read_csv("")

,Unnamed: 0
count,768457.000000
mean,384228.000000
std,221834.572242
min,0.000000
25%,192114.000000
50%,384228.000000
75%,576342.000000
max,768456.000000
